In [39]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

['cleandata', 'jigsaw-unintended-bias-in-toxicity-classification']


In [40]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
allWords = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            allWords.append(temp1)
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [41]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [42]:
x = baseTrain
y = labels

In [43]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TFIDF = vectorizer_TF.fit_transform(x_min)
#X_TestTFIDF = vectorizer_TF.transform(baseTest)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer()
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [53]:
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
#clf = svm.SVR()
#clf = linear_model.LinearRegression()
#clf = DecisionTreeRegressor(max_depth=10)
#clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')
#clf = GaussianNB()
#clf = MultinomialNB()
#clf = BernoulliNB()
clf = SGDClassifier(loss='log', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)

#Classificação com probabilidade
y_train_class = []
for t in range(len(y_train)):
    if(y_train[t]<0.5):
        y_train_class.append(0)
    else:
        y_train_class.append(1)

clf.fit(X_train,y_train_class)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=None, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [54]:
from sklearn import metrics
y_pred = clf.predict(X_test) 
y_pred_prob = clf.predict_proba(X_test) 
print(y_pred_prob)
print(X_test.shape)

y_pred_class = y_pred
y_test_class = []
y_pred_prob_tox = []


for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
    y_pred_prob_tox.append(y_pred_prob[t][0])

mse = metrics.mean_squared_error(y_test, y_pred_prob_tox)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(acc)
print(prec)
print(rec)
print(mse)

with open('results_validation.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Logistic Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

[[0.70261464 0.29738536]
 [0.71701575 0.28298425]
 [0.76914094 0.23085906]
 ...
 [0.6671354  0.3328646 ]
 [0.73540868 0.26459132]
 [0.77958236 0.22041764]]
(54011, 109767)
0.5077949409542871
0.7389420673566496
0.9911504424778761
0.015640273704789834
0.2095700138154107


In [ ]:
# from sklearn import metrics
# y_pred = clf.predict(X_test) 

# y_pred_class = []
# y_test_class = []

# for t in range(len(y_test)):
#     if(y_test[t]<0.5):
#         y_test_class.append(0)
#     else:
#         y_test_class.append(1)
#     if(y_pred[t]<0.5):
#         y_pred_class.append(0)
#     else:
#         y_pred_class.append(1)

# mse = metrics.mean_squared_error(y_test, y_pred)
# auc = metrics.roc_auc_score(y_test_class, y_pred_class)
# acc = metrics.accuracy_score(y_test_class, y_pred_class)
# prec = metrics.precision_score(y_test_class, y_pred_class)
# rec = metrics.recall_score(y_test_class, y_pred_class)

# print(auc)
# print(mse)
# print(acc)
# print(prec)
# print(rec)

# with open('results_validation.csv', mode='w') as employee_file:
#     writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     writer.writerow(["Linear Regression"])
#     writer.writerow(["mse: ", mse])
#     writer.writerow(["auc: ", auc])
#     writer.writerow(["accuracy: ", acc])
#     writer.writerow(["precision: ", prec])
#     writer.writerow(["recall: ", rec])

In [50]:
y_pred_test = clf.predict_proba(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        print(ids[id])
        print(y_pred_test[id][1])
        writer.writerow([ids[id], y_pred_test[id][1]])

7000000
0.3723297225579444
7000001
0.004770431709031489
7000002
0.10694897675968629
7000003
8.265933179082736e-08
7000004
0.996364199146502
7000005
0.11663861486719174
7000006
0.7397694452926724
7000007
0.29356563350217935
7000008
9.05505319347157e-07
7000009
0.0005901335091007561
7000010
0.734739679001426
7000011
0.9011837092314456
7000012
0.16846553845654813
7000013
0.0004368164789466977
7000014
0.0016020657666342116
7000015
0.0013247973145129352
7000016
0.006514672333997879
7000017
2.3294872169063983e-07
7000018
0.00021096230289765886
7000019
0.27514444636973967
7000020
0.9432435899665875
7000021
0.07922114107795453
7000022
0.06910097697860748
7000023
0.725255688875073
7000024
0.9211081178672959
7000025
0.6887614956079764
7000026
0.19897231289214695
7000027
0.02023143710499251
7000028
0.8226084970208608
7000029
5.526077516660411e-06
7000030
0.001331273875271943
7000031
0.00033775187583796754
7000032
0.8591588630082975
7000033
0.002568558940161046
7000034
0.5043818237406678
7000035
0

7000963
0.6022006754478725
7000964
0.041921510880705844
7000965
0.00012447698408508354
7000966
0.7778803785481485
7000967
0.8820417505348946
7000968
0.735468638641057
7000969
0.6558864163789612
7000970
0.02421652348992781
7000971
0.211445706328496
7000972
0.040135854418470285
7000973
1.0219082112427526e-05
7000974
0.8454455614452162
7000975
0.017707311763875487
7000976
0.012812555522532148
7000977
0.9317879985027223
7000978
0.6554261431125838
7000979
0.4349024996973959
7000980
0.6689850043256536
7000981
0.018560226393038923
7000982
0.9514861985847377
7000983
1.338607241491595e-05
7000984
0.13580187436722477
7000985
0.00028393569721896976
7000986
0.8963842186174189
7000987
0.15659410554301076
7000988
0.08736256489235063
7000989
0.8834648012918724
7000990
0.7022538721222563
7000991
0.3015543581436005
7000992
0.010940224772323273
7000993
0.016818652499341554
7000994
0.953695463698597
7000995
0.918907955230177
7000996
0.9540277833118861
7000997
0.9893496717249552
7000998
0.2090315610813449

0.7300090506550929
7001835
0.6484400543787724
7001836
2.5690480588636126e-07
7001837
3.213464028040763e-08
7001838
1.9318408421339106e-07
7001839
0.45940493971925506
7001840
1.1671891331016615e-08
7001841
0.00021976370188630173
7001842
0.003594518866689092
7001843
0.0006457395943843479
7001844
0.9170525090862094
7001845
5.012882649825373e-06
7001846
0.8505162291148171
7001847
0.6932058240178446
7001848
0.9083783305338338
7001849
0.7446244158698805
7001850
0.6289567769509148
7001851
0.647175434725183
7001852
0.1603478338708301
7001853
0.0012580291871515828
7001854
0.7708240622867872
7001855
0.8335220963562785
7001856
3.884170968746853e-09
7001857
0.6854851428665457
7001858
0.8411920884107856
7001859
0.0034860315652064104
7001860
3.871762565703302e-11
7001861
0.0001411249052933845
7001862
0.7440244033528131
7001863
0.8892608233324781
7001864
0.8715123152666138
7001865
0.004531779795901832
7001866
0.8718222559537764
7001867
0.11496555021860863
7001868
0.8542793234869892
7001869
0.04216806

0.8043748103432685
7002713
0.4355842968553713
7002714
0.7677313410187802
7002715
4.198793441282689e-05
7002716
0.007726915136516718
7002717
0.1530119027915449
7002718
0.09347756596688746
7002719
0.6167223871223575
7002720
0.020071941243799775
7002721
7.155694264273386e-07
7002722
9.831759497669706e-12
7002723
0.10209486078531067
7002724
0.11070642222834653
7002725
1.4238570807162426e-05
7002726
0.7472552475078137
7002727
0.0278464475410806
7002728
0.3094408252879877
7002729
2.8279276846922663e-05
7002730
0.21383995152674667
7002731
0.8691576747253748
7002732
0.9047755340215311
7002733
0.8254569606927331
7002734
0.6168911759091402
7002735
0.8665099346195608
7002736
0.2841991988526617
7002737
5.225347031727226e-09
7002738
0.9312918169973107
7002739
0.7288830730967143
7002740
3.599247393906208e-06
7002741
0.7527861706015779
7002742
0.5587389542981982
7002743
0.9203887239044776
7002744
0.050449910994123645
7002745
0.9021101969851435
7002746
0.0004441671848511633
7002747
3.925486273139815e-

7003571
0.010681924471842394
7003572
7.44033411600135e-09
7003573
0.8825931752006814
7003574
8.842096877582906e-07
7003575
0.00029363594349123
7003576
7.827620153163244e-05
7003577
0.6286730023399234
7003578
0.033735588258552565
7003579
0.9411619594633004
7003580
0.00010375593079503341
7003581
0.7978427189211679
7003582
0.3284455428142874
7003583
0.35722621453429754
7003584
0.8601153297087013
7003585
0.6058568409040525
7003586
0.08450874868308618
7003587
0.6490273906190471
7003588
0.856319148116865
7003589
0.001390646462580831
7003590
0.013915187443811514
7003591
0.49400009144453794
7003592
0.9167776064425585
7003593
0.21219029356737934
7003594
0.09922797036564963
7003595
0.6892577971646163
7003596
8.370042843718169e-08
7003597
0.004223829323200939
7003598
0.15697179118175875
7003599
0.10371860956259855
7003600
8.104778973158562e-07
7003601
0.8401906707662261
7003602
0.17692724174263927
7003603
0.02586326011870344
7003604
0.6847869200246063
7003605
0.00011076243960917959
7003606
0.8717

...

7093708
0.057158981851072016
7093709
0.3986678881743001
7093710
0.6267471722847586
7093711
0.3578680990859622
7093712
0.7352838707077796
7093713
4.6471789526166236e-05
7093714
0.20055575410571946
7093715
0.9206983860817434
7093716
2.621707744385172e-09
7093717
0.8781039339568281
7093718
0.0011835077696584345
7093719
0.0006358827897167119
7093720
0.09340175116805231
7093721
0.006985445344034868
7093722
0.11431456664672189
7093723
0.860394426174567
7093724
3.226934290562204e-05
7093725
0.0007535715278938374
7093726
0.9308046228194741
7093727
0.955314039202452
7093728
0.28955301642798337
7093729
0.8185963368959027
7093730
0.7701518103461412
7093731
0.9176150075520522
7093732
0.618751333304024
7093733
1.1037159673467583e-06
7093734
0.0071416159226220235
7093735
0.0018076957714030914
7093736
1.4305141802248163e-08
7093737
0.03403250207559432
7093738
0.13905106503974052
7093739
0.16574197773273588
7093740
0.7120639384841515
7093741
0.13311436535895516
7093742
0.08107120672633732
7093743
0.85

0.8982681399943698
7094556
0.33438944502802903
7094557
0.7253221655151202
7094558
2.645367540888595e-06
7094559
0.0007214207371421255
7094560
0.8421099798159515
7094561
0.006643023494121596
7094562
0.4289000021104976
7094563
0.8860454762421839
7094564
6.180452902830755e-06
7094565
0.6004110343901382
7094566
0.2890808238177895
7094567
0.5892415155710155
7094568
0.9061904142360787
7094569
0.38041594347203195
7094570
0.20298043099883437
7094571
0.15532146817699996
7094572
0.9001577760311114
7094573
0.4775872650778022
7094574
0.06898259917097475
7094575
0.3590892641238024
7094576
0.7277374270172736
7094577
2.0859542682167902e-05
7094578
0.3985168226986507
7094579
0.40740011083315
7094580
0.30156933534156405
7094581
0.01615640658538722
7094582
1.5927452054425695e-09
7094583
0.4962097816755288
7094584
0.9409884093162306
7094585
0.0009672801107172926
7094586
0.8506104086753821
7094587
0.07124255927593844
7094588
0.7839394417565464
7094589
0.3395950486844119
7094590
0.99971931830043
7094591
0.

0.020215988798455813
7095566
0.4095794542306802
7095567
0.002708408286506396
7095568
0.7518062202964061
7095569
0.8666121007719554
7095570
7.961943694591212e-07
7095571
0.8621026762335424
7095572
0.020921461622126148
7095573
0.12845479187191527
7095574
6.20408000209712e-05
7095575
2.7121304841408732e-11
7095576
0.35941043022170704
7095577
0.7587787512152471
7095578
0.8798621269510404
7095579
0.49964327779933176
7095580
0.9915854980294663
7095581
0.7664667718821342
7095582
0.5576675446143402
7095583
0.0010851919376981
7095584
0.03613436052996228
7095585
0.003340725437699263
7095586
0.983065744247185
7095587
0.8942355731581543
7095588
0.0009136448951969131
7095589
0.6066284633639502
7095590
0.9020506640902473
7095591
0.7997525742832688
7095592
0.0032362500466486224
7095593
0.5025899695835786
7095594
0.9376559604050801
7095595
2.0626981888323257e-09
7095596
0.06779507308817029
7095597
0.7677217021507107
7095598
0.5439380450403336
7095599
0.7653864671754327
7095600
0.8356638104908861
70956

7096434
0.072746888228598
7096435
0.4510394204620682
7096436
5.911203789920404e-06
7096437
0.00015413864746500702
7096438
0.03578586928377268
7096439
6.646686275316935e-09
7096440
0.9998544673962061
7096441
0.6441614941760471
7096442
0.8551977672081185
7096443
0.5909000445539575
7096444
0.9280638240706687
7096445
0.13685301191828184
7096446
6.4503264610279e-11
7096447
9.16463231688774e-05
7096448
0.015841429646914613
7096449
0.823175249598101
7096450
0.7279802393436171
7096451
2.3447285319362632e-06
7096452
0.0412156729361364
7096453
0.05745973874243899
7096454
0.012144252708481233
7096455
1.882836043901124e-10
7096456
0.0015439704187279139
7096457
0.5658753897476486
7096458
1.2745429527623026e-07
7096459
0.05994392808560724
7096460
0.9153503300992603
7096461
0.029594364985011577
7096462
0.1724854025028147
7096463
0.8399469117576721
7096464
0.0024316504164774936
7096465
4.498035774703149e-09
7096466
0.013106096135414249
7096467
0.019025866393267206
7096468
0.0015753712999556288
7096469

0.25123305914483696
7097217
1.1548447580510097e-08
7097218
0.8463480774640816
7097219
0.8882888733188488
7097220
0.8590750727865547
7097221
0.6221987462319806
7097222
6.860076653484184e-10
7097223
0.6699113501960425
7097224
0.0582850300757999
7097225
0.2689504027656995
7097226
0.0035492407362750352
7097227
0.15652282610892018
7097228
0.5568545988259557
7097229
0.4980072389188757
7097230
0.5051447849204516
7097231
2.390177027659297e-10
7097232
2.601341445577816e-10
7097233
0.028808725637167937
7097234
2.035867162181478e-05
7097235
0.7664576791645809
7097236
1.6343057019841132e-05
7097237
0.7590810579962705
7097238
0.0001034789920071114
7097239
0.5636967945874439
7097240
0.18967598524988433
7097241
0.9153503300992603
7097242
0.14910491846671134
7097243
0.48711143266811113
7097244
0.0008353180302732377
7097245
0.10656940152341636
7097246
0.979441702213285
7097247
0.38995534760302814
7097248
0.5210599368748754
7097249
0.8375845516936992
7097250
0.9788480009768201
7097251
0.3110630606475762